# Importo librerias

In [1]:
import numpy as np 
import pandas as pd 
# import cv2 # Para cargar/mostrar/procesar imagenes
import json
import glob # Para listar los archivos en una carpeta

# Librerias propias (contienen los modelos)
import model_metadata
import model_sentiment



ModuleNotFoundError: No module named 'model_metadata'

# Codigo Auxiliar

In [53]:
#Para borrar la cache de las librerias propias
#import importlib
#importlib.reload(model_sentiment)
#importlib.reload(model_metadata)

NameError: name 'ProfileReport' is not defined

# Cargo archivos

## Primer analisis:

Cargo el dataset completo y para el ID **0a0e8c15b** cargo las imagenes que tiene, metadata de las imagenes y analisis de sentimiento de la descripcion.

In [34]:
#No ejecutar
# Cargar CSV train de carpeta data/train/train.csv
train_data = pd.read_csv('../data/train/train.csv', index_col='PetID')
train_image_1 = cv2.imread('../data/train_images/0a0e8c15b-1.jpg')
train_image_2 = cv2.imread('../data/train_images/0a0e8c15b-2.jpg')
with open('../data/train_metadata/0a0e8c15b-1.json', 'r') as archivo:
    # Carga el contenido del archivo JSON en un diccionario
    train_metadata_1 = json.load(archivo)
with open('../data/train_metadata/0a0e8c15b-2.json', 'r') as archivo:
    # Carga el contenido del archivo JSON en un diccionario
    train_metadata_2 = json.load(archivo)
with open('../data/train_sentiment/0a0e8c15b.json', 'r') as archivo:
    # Carga el contenido del archivo JSON en un diccionario
    train_sentiment = json.load(archivo)

Creo una funcion para importar los archivos json a una lista. Los archivos los modelo y los inserto en una lista.

In [2]:
def getModel(model, listFileFullPath):
    '''model = clase del modelo que se quiere cargar (objeto)
    listFullPath = ruta completa de los archivos a cargar (List[string])
    '''
    #f_train_metadata = glob.glob('../data/train_metadata/*.json')
    #f_train_sentiments = glob.glob('../data/train_sentiment/*.json')

    listObject = []
    for file in listFileFullPath:
        
        file_json = None
        objectModel = None
        
        with open(file, 'r') as archivo:
            file_json = json.load(archivo)    

        fileName = file.split('/')[-1]
        file_json['fileName'] = fileName
        file_json['PetID'] = fileName.split('.')[0].split('-')[0]
        objectModel = model.Model(**file_json)
        listObject.append(objectModel)

    return listObject 

In [3]:
train_data = pd.read_csv('../data/train/train.csv', index_col='PetID')

In [4]:
#train_data = pd.read_csv('../data/train/train.csv', index_col='PetID')
f_train_metadata = glob.glob('../data/train_metadata/*.json')
f_train_sentiments = glob.glob('../data/train_sentiment/*.json')

list_metadata = getModel(model_metadata, f_train_metadata)
list_sentiments = getModel(model_sentiment, f_train_sentiments)

del f_train_metadata
del f_train_sentiments

## Unifico dataset e informacion extra

En esta seccion vamos a agregar informacion exta al dataset original. Vamos a agregar las siguientes variables:

De la imagen:

1. Imagen? [SI/NO] Si la mascata tiene imagen 1 sino 0.
2. ImagenCantidad Cantidad de imagenes que posee la mascota

Del Comentario:
1. magnitude
2. score
3. Cantidad de sentencias
4. EntidadCantidad
5. Entidad por tipo (son varias columnas)
6. El nombre de la mascota figura como entidad?

Datos descriptivos:
1. Agregar Beed
2. Agregar Color
3. Agregar PetFinder
4. Agregar state


In [12]:
len(list_metadata) #58311
len(list_sentiments) #14442

list_metadata[10].fileName


'924cc5438-2.json'

In [5]:
getPetID = lambda x: x.PetID
getPetID_vec = np.vectorize(getPetID)

In [6]:
#Si tiene o no imagen
train_data['Imagen?'] = train_data.index.isin(getPetID_vec(list_metadata))
train_data['sentiment?'] = train_data.index.isin(getPetID_vec(list_sentiments))

In [ ]:
# Para validar (no ejecutar)
#Buscar archivo por linea de comando
#find . -type f -name "*86e1089a3*"
#find . -type f -name "*1bc0f89d8*"
#1bc0f89d8 
#train_data[train_data['Imagen?'] == False].head(10)
train_data[train_data['sentiment?'] == True].head(10)


In [7]:
#Busco la cantidad de imagenes y las agrego al DF original
df = pd.DataFrame(getPetID_vec(list_metadata))
df = df[0].value_counts()
df.columns = ['PetID', 'CountImagen']
train_data = train_data.join(df, on='PetID', how='left', lsuffix='_left', rsuffix='_right')
del df

In [ ]:
train_data.head()

In [45]:
#1. magnitude
#2. score
#3. Cantidad de sentencias
#4. EntidadCantidad

getScore = lambda x: x.documentSentiment.score
getScore_vec = np.vectorize(getScore)

getMagnitude = lambda x: x.documentSentiment.magnitude
getMagnitude_vec = np.vectorize(getMagnitude)

getLenEntity = lambda x: len(x.entities)
getLenEntity_vec = np.vectorize(getLenEntity)

getLenSentences = lambda x: len(x.sentences)
getLenSentences_vec = np.vectorize(getLenSentences)



In [46]:
df = pd.DataFrame(getScore_vec(list_sentiments), index=getPetID_vec(list_sentiments))
#df = df[0].value_counts()
df.columns = ['Score']
train_data = train_data.join(df, on='PetID', how='left', lsuffix='_left', rsuffix='_right')
del df


df = pd.DataFrame(getMagnitude_vec(list_sentiments), index=getPetID_vec(list_sentiments))
#df = df[0].value_counts()
df.columns = ['Magnitude']
train_data = train_data.join(df, on='PetID', how='left', lsuffix='_left', rsuffix='_right')
del df

df = pd.DataFrame(getLenEntity_vec(list_sentiments), index=getPetID_vec(list_sentiments))
#df = df[0].value_counts()
df.columns = ['CountEntity']
train_data = train_data.join(df, on='PetID', how='left', lsuffix='_left', rsuffix='_right')
del df

df = pd.DataFrame(getLenSentences_vec(list_sentiments), index=getPetID_vec(list_sentiments))
#df = df[0].value_counts()
df.columns = ['CountSentences']
train_data = train_data.join(df, on='PetID', how='left', lsuffix='_left', rsuffix='_right')
del df



In [ ]:
#from pandas_profiling import ProfileReport
# prof = ProfileReport(train_data, title="Pandas Profiling Report", explorative=True)
# prof.to_file(output_file='../output/output.html')

from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report")
profile.to_file(output_file='../output/output.html')


In [ ]:
#Analisis
#Age ==> outliers valor=255


In [ ]:
#import dtale
#dtale.show(train_data)
#http://localhost:40000/
